In [ ]:
%load_ext autoreload
%autoreload 2

from data_prep import DataPreparation, filtering_data
from lstm import LSTMModelTrainerAttention, LSTMRollingForecaster
from rf import RandomForestTrainer
from lgbm import LightGBMTrainer
from catboost_trainer import CatBoostTrainer
from transformer import TransformerModelTrainer
import os
import dill as pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import warnings
warnings.simplefilter(action='ignore')
warnings.filterwarnings("ignore", category=UserWarning)


In [ ]:
output= r'C:\Users\iLabs_6\Documents\Tex\allwithCBAS'
modelname = 'extended_df_ATOT-jorn2'
modelname = 'extended_df_ATOT'
output= r'C:\Users\iLabs_6\Documents\Tex\realtimetest3'

modelname = 'newbaseline_atot2'
# output= r'C:\Users\iLabs_6\Documents\Tex\allwithCBAS'
# modelname = 'extended_df_ATOT'
 
with  open(os.path.join(output, modelname), 'rb') as f:  
    extended_df = pickle.load(f)

extended_df['cap_DEP'].fillna(extended_df['cap_DEP'].mean(), inplace=True)
extended_df['cap_DES'].fillna(extended_df['cap_DES'].mean(), inplace=True)
extended_df = extended_df.loc[:, ~extended_df.columns.str.contains('t_to_eobt', case=False)]
extended_df = extended_df.loc[:, ~extended_df.columns.str.contains('t_to_atot', case=False)]
# extended_df = extended_df.drop_duplicates()
# print(f'{extended_df.describe()=}')
pd.set_option("display.max_columns", None)
X, y, colnames = filtering_data(extended_df, airport ='EHAM', save=True)
X = X.drop(['ADES_EHAM', 'ADESLong', 'ADESLat', 'cbasentry'], axis =1)
# X.describe()

# output= r'C:\Users\iLabs_6\Documents\Tex\realtimetest3'
modelname = 'extended_df_ETOT'
modelname = 'newbaseline_etot2'


with  open(os.path.join(output, modelname), 'rb') as f:  
    extended_real= pickle.load(f)
extended_real = extended_real.loc[:, ~extended_real.columns.str.contains('t_to_eobt', case=False)]
extended_real = extended_real.loc[:, ~extended_real.columns.str.contains('t_to_atot', case=False)]
# extended_real = extended_real.drop_duplicates()

pd.set_option("display.max_columns", None)
# print(f'{extended_real}')
X_r, y_r, colnames = filtering_data(extended_real, airport ='EHAM', save=False)
X_r = X_r.drop(['ADES_EHAM', 'ADESLong', 'ADESLat', 'cbasentry'], axis =1)
# X.fillna(0, inplace=True)
# print(f'xxxxx = {X_r}')

In [ ]:
dform = "%Y-%m-%d %H:%M:%S"

ext = extended_real.assign(EOBT=lambda x: pd.to_datetime(x.EOBT, format=dform)).sort_values(by='EOBT')
total_rows = len(ext)
last_20_percent_rows = int(total_rows * 0.2)

# Select the last 20% of rows
last_20_percent = ext.sort_values(by='EOBT').iloc[-last_20_percent_rows:]

# Display the description of the last 20%
last_20_percent.describe()


In [ ]:
data_prep = DataPreparation()


X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, time_horizons, cbaslabels = data_prep.fit_transform_data(X, y)
X_real, ETOT_horizons, cbaslabels = data_prep.transform_data(X_r)

X_train, X_test, y_train, y_test = data_prep.fit_transform_data(X, y, split_ratio=0.8, mode='rf')
print(f'{X_train_tensor.shape=}')
print(f'{X_test_tensor.shape=}')



In [ ]:
model_trainer = LSTMModelTrainerAttention(data_prep=data_prep, input_size=data_prep.input_size, model_type='varattention')
print(f'{X_train_tensor.shape=}')
print(f'{X_test_tensor.shape=}')
# Set data loaders
model_trainer.set_data_loaders(data_prep.train_loader, data_prep.test_loader, data_prep.time_steps)

# ANDERE SWITCH, afhankelijk van eerste prediction
best_model = model_trainer.hyperparameter_search(1)
#2707 16 it
# Evaluate the best model and plot the results
y_pred, y_test = model_trainer.evaluate_and_plot(best_model, X_test_tensor, y_test_tensor, data_prep.scaler_y, time_horizons)



In [ ]:
y_pred, y_test = model_trainer.evaluate_and_plot(best_model, X_test_tensor, y_test_tensor, data_prep.scaler_y, time_horizons)


In [ ]:
output= r"C:\Users\iLabs_6\Documents\Tex\AirTrafficDelays\LSTM_Models"

modelname = 'newbaseline-s6'
with open(os.path.join(output, modelname), 'wb') as f:  
    pickle.dump(best_model, f, protocol=pickle.HIGHEST_PROTOCOL)
    print(f"FlightDataset has been saved successfully. {modelname}")

with  open(os.path.join(output, modelname), 'rb') as f:  
    best_model = pickle.load(f)


In [ ]:
# Initialize RandomForestTrainer with desired hyperparameters
trainer_rf = RandomForestTrainer(dataprep=data_prep, horizons=data_prep.time_horizons)


# # Share the scaler_y between data_prep_rf and trainer_rf
trainer_rf.scaler_y = data_prep.scaler_y

# # Train the model
trainer_rf.train(X_train, y_train)

In [ ]:
trainer_rf.evaluate(X_test, y_test)

In [ ]:
trainer_gbm = LightGBMTrainer(dataprep=data_prep,n_estimators=20, max_depth=10, min_samples_split=5, horizons=data_prep.time_horizons)

# Share the scaler_y between data_prep_rf and trainer_rf
trainer_gbm.scaler_y = data_prep.scaler_y

# Train the model
trainer_gbm.train(X_train, y_train)

In [ ]:
trainer_gbm.evaluate(X_test, y_test)

In [ ]:
trainer_cat = CatBoostTrainer(dataprep=data_prep, horizons=data_prep.time_horizons)

# Share the scaler_y between data_prep_rf and trainer_rf
trainer_cat.scaler_y = data_prep.scaler_y

# Train the model
trainer_cat.train(X_train, y_train)

In [ ]:
trainer_cat.evaluate(X_test, y_test)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from transformer import TransformerModelTrainer
import torch


transformer_trainer = TransformerModelTrainer(
    dataprep=data_prep,
    horizons=[x for x in range(0,305,5)][::-1],
    device='cuda' if torch.cuda.is_available() else 'cpu',
    model_folder='transformer_models',
    d_model=128,
    nhead=8,
    num_encoder_layers=4,
    dim_feedforward=256,
    dropout=0.1,
    activation='relu',
    learning_rate=1e-3,
    batch_size=32,
    num_epochs=10,
    early_stopping_patience=5
)

# Train the Transformer models
transformer_trainer.train(X_train_tensor, y_train_tensor, X_val=None, y_val_dict=None)

In [ ]:
from transformer import TransformerModelTrainer, TransformerRollingForecaster


transformer_trainer = TransformerModelTrainer(data_prep.input_size, data_prep, model_type='Transformer', log_dir='runs/transformer')

transformer_trainer.set_data_loaders(data_prep.train_loader, data_prep.test_loader, data_prep.time_steps)

# ANDERE SWITCH, afhankelijk van eerste prediction
best_transformer_model = transformer_trainer.hyperparameter_search(1)
#2707 16 it
# Evaluate the best model and plot the results
y_pred, y_test = transformer_trainer.evaluate_and_plot(best_transformer_model, X_test_tensor, y_test_tensor, data_prep.scaler_y, time_horizons)


In [ ]:
output= r"C:\Users\iLabs_6\Documents\Tex\AirTrafficDelays\Transformer_Models"

modelname = 'Transformer_nb7_198.pkl'
with open(os.path.join(output, modelname), 'wb') as f:  
    pickle.dump(best_transformer_model, f, protocol=pickle.HIGHEST_PROTOCOL)
    print(f"FlightDataset has been saved successfully. {modelname}")

with  open(os.path.join(output, modelname), 'rb') as f:  
    best_transformer_model = pickle.load(f)

y_pred, y_test = transformer_trainer.evaluate_and_plot(best_transformer_model, X_test_tensor, y_test_tensor, data_prep.scaler_y, time_horizons)
y_pred-y_test

In [ ]:
# from transformer import TransformerModelTrainer, TransformerRollingForecaster
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize lists for errors and predictions
abs_ap = []  # Absolute errors for MAE
ap = []
squared_errors = []  # Squared errors for RMSE
y_true = []  # True values
y_pred = []  # Predicted values
trans_ap = []
target_length = 61  # Target length for each error array
start_idx, end_idx = -1000, -100  # Index range
start_idx, end_idx =int( 0.8*len(y_r)), -1  #all!

# start_idx, end_idx = 56455, -1     # Uncomment to use all indices

for fnr in tqdm(np.where((y_r <= 130))[0][start_idx:end_idx], desc="Processing Transformer Forecasts"):
    # Create rolling forecaster instance
    recursive = TransformerRollingForecaster(best_transformer_model, data_prep, X_real[fnr], ETOT_horizons)
    pred = recursive.rolling_forecast()

    # Compute absolute and squared errors per timestep
    absolute_error_per_timestep = np.abs(pred[-target_length:] - y_r[fnr])
    squared_error_per_timestep = (pred[-target_length:] - y_r[fnr]) ** 2
    error_per_timestep = (pred[-target_length:] - y_r[fnr])

    # Ensure each error array has `target_length` elements by padding with NaNs if needed
    if len(absolute_error_per_timestep) < target_length:
        pad_length = target_length - len(absolute_error_per_timestep)

        padded_abs_error = np.pad(absolute_error_per_timestep, (0, pad_length), constant_values=np.nan)
        padded_error = np.pad(error_per_timestep, (0, target_length - len(error_per_timestep)), constant_values=np.nan)
        
        padded_sq_error = np.pad(squared_error_per_timestep, (0, pad_length), constant_values=np.nan)
        abs_ap.append(padded_abs_error)
        ap.append(padded_error)

        squared_errors.append(padded_sq_error)
    else:
        abs_ap.append(absolute_error_per_timestep)
        squared_errors.append(squared_error_per_timestep)
        ap.append(error_per_timestep)


    # Handle y_true and y_pred for R² calculation
    true_values = y_r[fnr] * np.ones(target_length)  # Repeat the true value
    predicted_values = pred[-target_length:]

    if len(predicted_values) < target_length:
        pad_length = target_length - len(predicted_values)
        y_true.append(true_values)
        y_pred.append(np.pad(predicted_values, (0, pad_length), constant_values=np.nan))
    else:
        y_true.append(true_values)
        y_pred.append(predicted_values)

# Convert lists to NumPy arrays for easier manipulation
abs_ap = np.array(abs_ap)  # Shape: (num_samples, target_length)
squared_errors = np.array(squared_errors)  # Shape: (num_samples, target_length)
y_true = np.array(y_true)  # Shape: (num_samples, target_length)
y_pred = np.array(y_pred)  # Shape: (num_samples, target_length)

# Compute per-timestep metrics
mae_per_timestep = np.nanmean(abs_ap[-61:], axis=0)
rmse_per_timestep = np.sqrt(np.nanmean(squared_errors[-61:], axis=0))
std_per_timestep = np.nanstd(abs_ap[-61:], axis=0)

# Compute overall metrics
overall_mae = np.nanmean(abs_ap)
overall_rmse = np.sqrt(np.nanmean(squared_errors))

# Flatten arrays for overall R² calculation
flat_y_true = y_true.flatten()
flat_y_pred = y_pred.flatten()
valid_mask = ~np.isnan(flat_y_true) & ~np.isnan(flat_y_pred)

overall_std = np.nanstd(abs_ap)


if np.any(valid_mask):
    overall_r2 = r2_score(flat_y_true[-61:], flat_y_pred[-61:])
else:
    overall_r2 = np.nan

# Print raw values
print("Raw True Values (y_true):", y_true)
print("Raw Predictions (y_pred):", y_pred)

# Print overall metrics
print(f"Overall MAE: {overall_mae:.4f}")
print(f"Overall RMSE: {overall_rmse:.4f}")
print(f"Overall R²: {overall_r2:.4f}")
print(f"Overall Std Deviation of MAE: {overall_std:.4f}")

# Plot MAE and RMSE per timestep
plt.figure(figsize=(10, 6))
plt.plot(ETOT_horizons[:target_length], mae_per_timestep, marker='o', label='MAE')
plt.plot(ETOT_horizons[:target_length], rmse_per_timestep, marker='x', label='RMSE')
plt.xlabel('Timestep')
plt.xticks(ETOT_horizons[:target_length], rotation='vertical')
plt.ylabel('Error')
plt.title('Transformer: MAE and RMSE for Each Timestep')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot mean error with standard deviation
mean_error_per_timestep = np.nanmean(y_true - y_pred, axis=0)
plt.figure(figsize=(10, 6))
plt.plot(ETOT_horizons[:target_length], mean_error_per_timestep, marker='o', label='Mean Error')
plt.fill_between(
    ETOT_horizons[:target_length],
    mean_error_per_timestep - std_per_timestep,
    mean_error_per_timestep + std_per_timestep,
    color='b',
    alpha=0.2,
    label='±1 Std Dev'
)
plt.xlabel('Timestep')
plt.xticks(ETOT_horizons[:target_length], rotation='vertical')
plt.ylabel('Mean Error')
plt.title('Transformer: Mean Error for Each Timestep with Standard Deviation')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()